In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles, make_classification

%load_ext autoreload
%autoreload 

### Dataset

In [8]:
X = np.random.randint(2, size = (2, 400)) # creo una matrice con due numeri (0,1), su 2 colonne e 5 linee
Y = np.logical_xor(X[0],X[1]) # creo delle coppie tra le due linee della matrice X (restituisci True se "or" is True)
Y = Y.reshape(1,X.shape[1])*1 # forziamo la macchina a capire che per l'output voglio una linea e 5 colonne. Moltiplico per 1 per trsaformare True et False en 0 et 1

#print(X)
print(f"X.shape = {X.shape}")
# print(Y)
print(f"Y.shape = {Y.shape}")

X.shape = (2, 400)
Y.shape = (1, 400)


### Structure du reseaux

In [9]:
n_x = int(X.shape[0]) # n° of inputs units
print(f'Input layer = {n_x} units (n_x)')
n_h1 = 3 # n° of neurones dans l'hidden layer 1
print(f'Hidden layer 1 = {n_h1} units (n_h1)')
n_h2 = 3 # n° of neurones dans l'hidden layer 2
print(f'Hidden layer 2 = {n_h2} units (n_h2)')
n_y = int(Y.shape[0]) # n° of output units
print(f'Output layer = {n_y} output (n_y)')
nn_structure = {'n_x':n_x,'n_h1':n_h1,'n_h2':n_h2,'n_y':n_y}
print()
print(f'NN structure = {nn_structure}')

Input layer = 2 units (n_x)
Hidden layer 1 = 3 units (n_h1)
Hidden layer 2 = 3 units (n_h2)
Output layer = 1 output (n_y)

NN structure = {'n_x': 2, 'n_h1': 3, 'n_h2': 3, 'n_y': 1}


### Initialisation des parametres

In [10]:
def init_params(n_x,n_h1,n_h2,n_y):
    
    '''
    Function qui inizialize mes parametres
    
    Weight initialisés random, Bias initialisés à zero
    
    '''
    
    np.random.seed(0)
    
    w1 = np.random.randn(n_h1,n_x)*0.01
    w2 = np.random.randn(n_h2,n_h1)*0.01
    w3 = np.random.randn(n_y,n_h2)*0.01
    b1 = np.zeros((n_h1,1))*0.01
    b2 = np.zeros((n_h2,1))*0.01
    b3 = np.zeros((n_y,1))*0.01
    
    params = {'w1':w1, 'w2':w2, 'w3':w3, 'b1':b1, 'b2':b2, 'b3':b3}
    
    print(f'w1.shape = {w1.shape}')
    print(f'w2.shape = {w2.shape}')
    print(f'w3.shape = {w3.shape}')
    print(f'b1.shape = {b1.shape}')
    print(f'b2.shape = {b2.shape}')
    print(f'b3.shape = {b3.shape}')
    
    return params

In [11]:
params = init_params(n_x,n_h1,n_h2,n_y)

w1.shape = (3, 2)
w2.shape = (3, 3)
w3.shape = (1, 3)
b1.shape = (3, 1)
b2.shape = (3, 1)
b3.shape = (1, 1)


### Forward Propagation

In [12]:
def sigmoid(z):
    
    '''
    Fonction d'activation sigmoid
    
    '''
    
    s = 1 / (1 + np.exp(-z))
    
    return s

In [13]:
def forward(X, params):

    '''
    Fonction de forward propagation
    
    On recupere les parametres initialisés pour calculer la fonction de pre-activation Z

    '''
    w1 = params['w1'] 
    w2 = params['w2']
    w3 = params['w3']
    b1 = params['b1']
    b2 = params['b2']
    b3 = params['b3']
    
    z1 = np.dot(w1,X)+b1
    a1 = sigmoid(z1)
    z2 = np.dot(w2,a1)+b2
    a2 = sigmoid(z2)
    z3 = np.dot(w3,a2)+b3
    a3 = sigmoid(z3)
    
    cache = {'z1':z1, 'z2':z2, 'z3':z3, 'a1':a1, 'a2':a2, 'a3':a3}
    
    print(f'a1.shape = {a1.shape}')
    print(f'a2.shape = {a2.shape}')
    print(f'a3.shape = {a3.shape}')
    
    return cache, a3

In [14]:
cache, a3 = forward(X, params)

a1.shape = (3, 400)
a2.shape = (3, 400)
a3.shape = (1, 400)


In [15]:
def cost_funct(A,Y):
    
    '''
    Cost function
    
    '''
    
    m = Y.shape[1]
    
    logprob = (Y * np.log(A) + (1-Y) * np.log(1-A))
    #ensuite la cost
    cost = -(np.sum(logprob))/m
    #je veux que l'on me retourne un nombre et non pas un array
    cost = np.squeeze(cost)
    #être sur que j'ai la cost au bon format
    assert(isinstance(cost,float))

    return cost

In [16]:
cost = cost_funct(a3,Y)
cost

0.6935222402338742

### Backward Propagation

In [17]:
def backward(X, Y, cache, params, l_r):
    
    a3 = cache['a3']
    a2 = cache['a2']
    a1 = cache['a1']
    w3 = params['w3']
    w2 = params['w2']
    w1 = params['w1']
    b1 = params['b1']
    b2 = params['b2']
    b3 = params['b3']
    
    m = Y.shape[1]
    
    dz3 = a3 - Y
    dw3 = 1/m * np.dot(dz3, a2.T)
    db3 = 1/m * np.sum(dz3, axis=1, keepdims=True)
    
    dz2 = (np.dot(w3.T, dz3)) * (1 - (a2**2))
    dw2 = 1/m * np.dot(dz2, a1.T)
    db2 = 1/m * np.sum(dz2, axis=1, keepdims=True)
    
    dz1 = (np.dot(w2.T, dz2)) * (1 - (a1**2)) 
    dw1 = 1/m * np.dot(dz1, X.T)
    db1 = 1/m * np.sum(dz1, axis=1, keepdims=True)
    
    w1 = params['w1'] - l_r * dw1
    b1 = params['b1'] - l_r * db1
    w2 = params['w2'] - l_r * dw2
    b2 = params['b2'] - l_r * db2
    w3 = params['w3'] - l_r * dw3
    b3 = params['b3'] - l_r * db3
    
    params['w1'] = w1
    params['b1'] = b1
    params['w2'] = w2
    params['b2'] = b2
    params['w3'] = w3
    params['b3'] = b3
    
    return params

In [20]:
backward(X, Y, cache, params, 1) # test backward function with learning rate = 1

{'w1': array([[ 0.01764692,  0.00401026],
        [ 0.00978732,  0.02240884],
        [ 0.01867581, -0.00977247]]),
 'w2': array([[ 0.00980086, -0.00121176, -0.00073411],
        [ 0.00399531,  0.00132908,  0.01443276],
        [ 0.00810193,  0.00171132,  0.00492707]]),
 'w3': array([[-0.04378321, -0.03233393, -0.04924615]]),
 'b1': array([[ 9.01102488e-06],
        [-9.01355561e-08],
        [ 3.22982644e-07]]),
 'b2': array([[ 0.00059503],
        [-0.00021954],
        [ 0.00097504]]),
 'b3': array([[-0.09407504]])}

In [22]:
def accuracy(Predictions, Labels):

    """
    Predictions shape (1, m)
    Labels shape (1, m)
    """
    
    n_correct = np.sum(Predictions == Labels)
    n_total   = Predictions.shape[1]
    
    accuracy = n_correct / n_total
    
    return accuracy

In [23]:
accuracy(Y, Y) # test accuracy function with labels & labels (doit retourner 1)

1.0